# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [4]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)

#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         done = env_info.local_done                         # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(done):                                   # exit loop if episode finished
#             break

#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
import torch
import numpy as np
from collections import deque
from MultiAgentDDPG import MultiAgentDDPG

agent = MultiAgentDDPG(state_size, action_size, num_agents)

In [6]:
def train(max_episodes = 50000, threshold = 0.5):

    # Initialize score variables
    scores = []
    scores_deque = deque(maxlen = 100)

    # Loop up to the maximum number of episodes. We will break out of this if we meet threshold early
    for iteration in range(1, max_episodes + 1):

        # Reset the environment for this training episode
        env_info = env.reset(train_mode = True)[brain_name]
        states = env_info.vector_observations
        agent.reset()
        score = np.zeros(num_agents)

        # Loop until an agent says it is complete for the episode
        while True:
            # Act on the environment
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations

            # Update the environment and agent
            rewards = env_info.rewards
            done = env_info.local_done
            agent.step(states, actions, rewards, next_states, done)
            score += rewards
            states = next_states

            # End the episode if any are reporting complete for this episode
            if np.any(done): 
                break

        # Update scores & average
        max_score = np.max(score) # We want the maximum score any agent got
        scores.append(max_score)
        scores_deque.append(max_score)
        average = np.mean(scores_deque) 

        # Periodically print status
        if iteration % 50 == 0:
            print('\rEpisode {}\tAverage: {:.2f}'.format(iteration, average))
        else:
            print('\rEpisode {}\tAverage: {:.2f}'.format(iteration, average), end="")

        # Early exit if we hit the target over the last 100 episodes
        if average >= threshold:
            print('\nTarget performance reached in {:d} episodes with an average score of {:.2f}'.format(iteration, average))
            break
        
    return scores

# Actually run the simulation
scores = train()

Episode 50	Average: 0.00
Episode 100	Average: 0.00
Episode 150	Average: 0.00
Episode 200	Average: 0.00
Episode 250	Average: 0.00
Episode 300	Average: 0.00
Episode 350	Average: 0.00
Episode 400	Average: 0.00
Episode 450	Average: 0.00
Episode 500	Average: 0.00
Episode 550	Average: 0.00
Episode 600	Average: 0.00
Episode 650	Average: 0.00
Episode 700	Average: 0.00
Episode 750	Average: 0.00
Episode 800	Average: 0.00
Episode 850	Average: 0.00
Episode 900	Average: 0.00
Episode 950	Average: 0.02
Episode 1000	Average: 0.02
Episode 1050	Average: 0.01
Episode 1100	Average: 0.00
Episode 1150	Average: 0.00
Episode 1200	Average: 0.00
Episode 1250	Average: 0.00
Episode 1300	Average: 0.01
Episode 1350	Average: 0.02
Episode 1400	Average: 0.02
Episode 1450	Average: 0.02
Episode 1500	Average: 0.03
Episode 1550	Average: 0.04
Episode 1600	Average: 0.04
Episode 1650	Average: 0.04
Episode 1700	Average: 0.07
Episode 1750	Average: 0.07
Episode 1800	Average: 0.06
Episode 1850	Average: 0.06
Episode 1900	Average:

In [ ]:
# Save the final agent actor / critic weights
for agent_index in range(num_agents):
    torch.save(agent.agents[agent_index].actor_local.state_dict(), 'snapshots/actor_{}.pth'.format(agent_index))
    torch.save(agent.agents[agent_index].critic_local.state_dict(), 'snapshots/critic_{}.pth'.format(agent_index))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

plt.show()

In [ ]:
env.close()